# Coursera Capstone project on Battle of neighbourhood

##### ~By Debapriyo Dasgupta

In [1]:
import numpy as np
import pandas as pd

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Data collection process

### Installing BeautifulSoup

In [3]:
!pip -q install requests
!pip -q install html5lib
!pip -q install bs4
print('Packages installed successfully!!!!')

Packages installed successfully!!!!


### Scraping website content

In [4]:
import requests
from bs4 import BeautifulSoup

In [5]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url)

soup = BeautifulSoup(req.content,'lxml')
#print(soup.prettify())

### Loading data in DataFrame

In [62]:
postal_table = soup.find("table", attrs={"class":"wikitable"})
postal_table_head = postal_table.tbody.find_all("tr")
postal_table_head

postal_data = []
for rows in postal_table_head:
    tr_row=[]
    for td in rows.find_all("td"):
        tr_row.append(td.text.replace('\n',' ').strip())
    postal_data.append(tr_row)
        
#Removing the first empty element
postal_data.pop(0)
#postal_data

#Converting the list to DataFrame
df_postal_code = pd.DataFrame(data = postal_data, columns=['PostalCode','Borough','Neighborhood'])
df_postal_code.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Cleaning the dataset
#### 1. Deleting the rows where borough is "Not assigned"

In [63]:
#Deleting the rows where borough is "Not assigned"
df_postal_code = df_postal_code[df_postal_code['Borough'] != 'Not assigned']
df_postal_code.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### 2. Merging different Neighborhoods grouped by PostalCode

In [64]:
postal_upd =df_postal_code.groupby(['PostalCode','Borough']).apply(lambda x: "%s" %', '.join(x.Neighborhood))
df_postal_upd = pd.DataFrame(postal_upd)
df_postal_upd.reset_index(inplace = True)

df_postal_upd.rename(columns={0:'Neighborhood'}, inplace=True)
df_postal_upd.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### 3. Replacing 'Not assigned' Neighborhood to corresponding Borough values

In [65]:
df_postal_upd.loc[df_postal_upd['Neighborhood']=='Not assigned', 'Neighborhood'] = df_postal_upd['Borough']

#To check if any further Neighborhood values are 'Not assigned'
df_postal_upd[df_postal_upd['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


### Finalizing the data after cleaning

In [66]:
#Final dataset
df_postal_can =df_postal_upd

#Fetching shape of final dataset
df_postal_can.shape

(103, 3)

# Fetching latitude longitude for each postal code

In [67]:
df_postal_can.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [14]:
#Importing geocoder
!pip -q install geocoder
import geocoder

print('Geocoder package installed and imported!!!')

Geocoder package installed and imported!!!


In [38]:
g=geocoder.google('Mountain View, CA')
print(g.latlng)

None


In [39]:
def getGeocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        
    return lat_lng_coords

In [50]:
#print(getGeocode('M1H'))
## This is going in an ever ending process. Hence using the spatial file

In [68]:
## Fetching the data from geospatial csv file
df_latlng=pd.read_csv('http://cocl.us/Geospatial_data')
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
## Merging the latitude longitude information with the dataset
df_postal_can.loc[df_postal_can['PostalCode']==df_latlng['Postal Code'],'Latitude']=df_latlng['Latitude']
df_postal_can.loc[df_postal_can['PostalCode']==df_latlng['Postal Code'],'Longitude']=df_latlng['Longitude']

In [70]:
df_postal_can.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
